In [1]:
import math

import numpy as np
from rdp import rdp
import time
from matplotlib import pyplot as plt
import pandas as pd
from os.path import join
import os
import math
from utilities import runQueryDTheta, loadData, donor_acceptor_dist
# %matplotlib widget

dataPath = r"C:\Users\ashamail\Box\Shared Materials_Abdullah_Hasan\Abdullah Data\allAtomsWithRings.csv"
savePath = r"C:\Users\ashamail\Desktop\drug-polymers\adbis extension\data\ellipsoids"
savepathDrive = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\adbis extension data"
oxygenPath = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\Text Data Files\flavanone_R1_os_0-200K.csv"
oxygen_location = r"C:\Users\ashamail\Desktop\data\os_200_1.txt"
nitrogenPath = r"C:\Users\ashamail\Desktop\data\nitrogen200k.txt"



In [2]:
data = pd.read_csv(dataPath)
# oxygen = pd.read_csv(oxygenPath, header=None)
# oxygen = oxygen.rename(columns={0: "x", 1: "y", 2: "z"})
oxygen = loadData(oxygen_location)
nitrogen = loadData(nitrogenPath)
min_z_points = pd.read_csv(join(savepathDrive, "min_z_points.csv"))

PSP = 44

In [5]:
def performRDP(data, epsilon):
    rdpData = rdp(data, epsilon=epsilon, return_mask=True)
    myData = []
    for i, truth in enumerate(rdpData):
        if truth:
            temp = list(np.append(data[i,:], i))
            myData.append(temp)
    myData = np.array(myData)
    return myData

In [6]:
# for all timesteps
# can also plot an instance using the line below
# plot_ell_and_plane(ell, points)

saveLocation = r"C:\Users\ashamail\OneDrive - Iowa State University\Molecular Simulation Data\adbis extension data\e_gatco + spmrdp"
d_deltas = np.arange(-6, 7, 1)
epsilons = [0.5, 0.75, 1]
countPerDelta = {}

toappend = np.arange(200000)
toappend = toappend.reshape(toappend.shape[0], 1)
rflOxygen = np.append(oxygen[:,0,:], toappend, axis = 1)

min_z_points["difference"] = min_z_points["min_z"].apply(lambda x: (x - PSP))
for epsilon in epsilons:
    for delta in d_deltas:
        s1 = time.time()
        countPerDelta[delta] = {}
        points_less_than_delta = min_z_points[min_z_points["difference"] < delta] #calculate ellipsoid and then calculate distance between min point and PSP using d_delte (1,2,3,4)
        # print(points_less_than_delta)

        # display(min_z_points[["min_z", "timestep"]].values)
        
        
        s1 = time.time()
        rdp_out = performRDP(points_less_than_delta[["min_z", "timestep"]].values, epsilon)
        timesteps = rdp_out[:, 2].astype(int)
        
        print(delta, epsilon, "pre-processing", time.time() - s1)

        # print(delta, epsilon, "done with pre processing, starting file saving")


        # temp = rflOxygen[timesteps]
        # np.savetxt(join(saveLocation, "rdp", str(epsilon), str(delta), "oxygen.csv"), temp, delimiter=",")
        # tempnitrogen = nitrogen[timesteps]
        # for i in range(216):
        # #     nitrogen[:,i,:].shape
        #     nitrogeni = np.append(tempnitrogen[:,i,:], timesteps.reshape(timesteps.shape[0], 1), axis = 1)
        #     np.savetxt(join(saveLocation, "rdp", str(epsilon), str(delta), str(i)+"-nitrogen.csv"), nitrogeni, delimiter=",")
        # print(delta, epsilon, "done with saving files, starting query")
        s = time.time()
        a, b = donor_acceptor_dist(oxygen[:,0,:], nitrogen, timesteps, hbond_threshold = 3.5 + 2*epsilon)
    # timesteps = runQueryDTheta(np.array(points_less_than_delta["timestep"]), oxygen, PSP) #for each d_delta instances, run the d_theta query
        print(delta, epsilon, "query processing", a.shape, time.time() - s)
        countPerDelta[delta][epsilon] = a

# for key, val in countPerDelta.items():
#     print(key, val.shape[0])

-6 0.5 pre-processing 62.618608236312866
-6 0.5 query processing (233,) 6.218013048171997
-5 0.5 pre-processing 78.33810758590698
-5 0.5 query processing (278,) 6.616071939468384
-4 0.5 pre-processing 78.22073197364807
-4 0.5 query processing (268,) 7.416030645370483
-3 0.5 pre-processing 84.81156730651855
-3 0.5 query processing (255,) 7.862591505050659
-2 0.5 pre-processing 86.62796592712402
-2 0.5 query processing (262,) 7.397063970565796
-1 0.5 pre-processing 86.39276146888733
-1 0.5 query processing (274,) 7.847095489501953
0 0.5 pre-processing 99.1748423576355
0 0.5 query processing (279,) 8.496103048324585
1 0.5 pre-processing 105.99398422241211
1 0.5 query processing (265,) 9.104076862335205
2 0.5 pre-processing 111.93320298194885
2 0.5 query processing (257,) 8.749048471450806
3 0.5 pre-processing 114.43373012542725
3 0.5 query processing (264,) 9.632084846496582
4 0.5 pre-processing 109.33412599563599
4 0.5 query processing (288,) 9.262107849121094
5 0.5 pre-processing 112.45

In [4]:
rdp_out.shape

(25879, 2)

In [44]:
saveDir = r"C:\Users\ashamail\Desktop\data\adbis extension\spkm_rdp_EGCTR_+-4"

for delta, val in countPerDelta.items():
    for epsilon, arr in val.items():
        path = join(saveDir, str(delta))
        if not os.path.exists(path):
            os.makedirs(path)
        np.save(join(path, str(epsilon)+".npy"), arr)
        

In [19]:
# plt.clf()
# plt_1 = plt.figure(figsize=(15, 5))
# plt.plot(min_z_points["timestep"][:100], min_z_points["min_z"][:100])
# plt.ylabel("min z coordinates")
# plt.xlabel("time")
# plt.show()

In [20]:
# # plt.clf()
# plt_2 = plt.figure(figsize=(15, 5))
# plt.plot(rdp_out[:, 1][:100], rdp_out[:, 0][:100])
# plt.ylabel("min z coordinates compressed")
# plt.xlabel("time")
# plt.show()

array([     0,      3,      5, ..., 199996, 199998, 199999])